# Combine Models From Multiple Runs With Model Averaging Ensemble

Deep learning neural network models are highly flexible nonlinear algorithms capable of learning a nearly infinite number of mapping functions. Frustration with this flexibility is the high variance in a final model. The same neural network model trained on the same dataset may nd one of many different possible good enough solutions each time it is run. In this tutorial, you will discover how to develop a model averaging ensemble in Keras to reduce the variance in a final model. Model averaging is an ensemble learning technique that reduces the variance in a final neural network model, sacrificing spread (and possibly better scores) in the model's performance for confidence in what performance to expect from the model. After completing this tutorial, you will know:

* Model averaging is an ensemble learning technique that can be used to reduce the expected variance of deep learning neural network models.
* How to implement model averaging in Keras for classification and regression predictive modeling problems.
* How to work through a multiclass classification problem and use model averaging to reduce the variance of the final model.

## Model Averaging Ensemble

Deep learning neural network models are nonlinear methods that learn via a stochastic training algorithm. This means that they are highly flexible, capable of learning complex relationships between variables and approximating any mapping function, given enough resources. A downside of this flexibility is that the models suffer high variance. This means that the models are highly dependent on the specific training data used to train the model and on the initial conditions (random initial weights) and serendipity during the training process. The result is a final model that makes different predictions each time the same model configuration is trained on the same dataset.

This can be frustrating when training a final model to make predictions on new data, such as operationally or in a machine learning competition. The high variance of the approach can be addressed by training multiple models for the problem and combining their predictions. This approach is called model averaging and belongs to a family of techniques called ensemble learning.

## Ensembles in Keras

## Model Averaging Ensemble Case Study